# Course overview

> Getting started

## Key Points

- The trick to using Stable Diffusion is to add artist names and consult [lexica.art](http://lexica.art)
- Stable Diffusion
  - Guidance scale modulates the degree of fidelity to the prompt
  - Img2Img allows the squishing of one image style into another
  - Fine-tuning:
      - Same vocab, [fine-tuned weights](https://lambdalabs.com/blog/how-to-fine-tune-stable-diffusion-how-we-made-the-text-to-pokemon-model-at-lambda)
      - New vocab, frozen weights: [Textual inversion](https://huggingface.co/docs/diffusers/training/text_inversion)
      - New vocab, fine-tuned weights: [Dreambooth](https://dreambooth.github.io)
  
## Advice

- GPUs might be worth investing in

## To-do

- [ ] unchecked

## Reading & Exploration

- https://colab.research.google.com/github/fastai/diffusion-nbs/blob/master/stable_diffusion.ipynb#scrollTo=lWwG1r9j5NnE
- https://lambdalabs.com/blog/how-to-fine-tune-stable-diffusion-how-we-made-the-text-to-pokemon-model-at-lambda

## Questions

- Proper notation for 
